### Import library

In [1]:
import re
import nltk
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import pandas as pd
import multiprocessing as mp
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
import math 
pd.__version__


'0.23.4'

### Import data

In [2]:
training = open("training_docs.txt","r",encoding = 'utf-8')
raw_train = []
for line in training:
    raw_train.append(line.lower())
    
label = open("training_labels_final.txt","r",encoding = 'utf-8')
train_label = []
for line in label:
    train_label.append(line.lower().strip("\n"))

stopword = open("stopwords_en.txt","r")
stopword_list = []
for line in stopword:
    stopword_list.append(line.lower().strip("\n"))

# Create basic list: ID_list, label_list, unique_label, content_list

In [3]:
ID_list = []
label_dict ={}
unique_label = []
content_list =[]
label_list = []
raw_train2 = list(filter(lambda x: x not in  ['\n','eod\n'], raw_train))

for i in range(len(raw_train2)):
    if i %2 != 0:
        content = raw_train2[i][5:].strip() # remove "TEXT" at beginning of the content
        if content != "'":
            content_list.append(content)
            ID_list.append(raw_train2[i-1][3:].strip())
        
        
for i in train_label:
    a = i.split()
    label_dict[a[0]]= a[1]
    
for i in label_dict.values():
    if i not in unique_label:
        unique_label.append(i)
for i in ID_list:
    label_list.append(label_dict[i])

In [4]:
corpus = pd.DataFrame({'id' : ID_list,
 'content' : content_list, 'label':label_list})

In [5]:
corpus.head()

,id,content,label
0,tr_doc_1,two german tourists have been found safe and w...,c1
1,tr_doc_2,act police have seized a rare drug during a ra...,c1
2,tr_doc_3,a 50-year-old brisbane man has been charged wi...,c1
3,tr_doc_4,in-depth discussions are continuing to resolve...,c1
4,tr_doc_5,homicide detectives are still questioning a ma...,c1


## Tokenize and lemmatize

In [6]:
from nltk.stem import WordNetLemmatizer

wordnet_lemmatizer = WordNetLemmatizer()


pattern = r"\w+(?:[-.@']\w+)*"

"""Define lemmatizer"""

def lemmatize_sent(text): # this do lemmatize and tokenise at the same time
# Text input is string, returns lowercased strings.
    return [wordnet_lemmatizer.lemmatize(word.lower()) 
            for word in nltk.regexp_tokenize(text,pattern)]
tokenizer = RegexpTokenizer(r"\w+(?:[-.@']\w+)*")

In [7]:
#optimised code
token_documents =[] # list token according to each documents
token_bag = [] # put all tokens together

token_chunk_lists = []
token_docs=[]
for raw_text in content_list:
    token_chunk_lists = lemmatize_sent(raw_text) #lemma
    token_chunk_lemma = []
    for token in token_chunk_lists:
        if token not in stopword_list and len(token) >= 3:
            token_chunk_lemma.append(token)
    token_docs.append(token_chunk_lemma)

corpus["token_docs"] = token_docs
unique_token_docs =[]
for i in token_docs:
    unique_token_docs.append(list(set(i)))
corpus["unique_token_docs"] = unique_token_docs




## tokenize and remove stopword and lemmatize only, then reconcate

In [8]:
#this is faster
corpus['nsw_token'] = corpus['token_docs'].apply(lambda x: ' '.join(x))

In [9]:
# corpus["nsw_token"] 
for i in corpus[corpus.id == 'tr_doc_1'].nsw_token: 
    print(i)

german tourist found safe spending hour lost rugged rainforest finch hatton gorge west mackay night area young mackay man fell jumped death week sergeant jon purcell rescuer located missing pair midnight aest


# modify

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfid_one_vectorizer = TfidfVectorizer(stop_words=stopword_list,tokenizer = tokenizer.tokenize,
                            ngram_range=(1, 1),max_df=0.95, min_df=0.0005,max_features = None
                            )
tfid_train = tfid_one_vectorizer.fit(corpus['nsw_token'])
tfid_bag_of_words = tfid_one_vectorizer.transform(corpus['nsw_token'])
tfid_bigram_bag = tfid_one_vectorizer.get_feature_names()
tfid_vector = tfid_one_vectorizer.fit_transform(corpus['nsw_token']).toarray()

In [11]:
tfid_one_gram_matrix = pd.DataFrame(tfid_vector,columns=tfid_bigram_bag)


In [12]:
a = tfid_one_gram_matrix.max()
one_gram_bag =[]
for i in range(len(a)):
    if a[i] >= a.quantile(.5):
        one_gram_bag.append(a.index[i])
len(one_gram_bag)

6653

### One gram

In [13]:
token_bag = [item for sublist in token_docs for item in sublist]
token_bag = list(set(token_bag))
one_gram = [item for sublist in unique_token_docs for item in sublist]

In [14]:
one_gram_doc =[]
for each_doc in corpus.unique_token_docs:
    element = list(set(each_doc).intersection(one_gram_bag))
    one_gram_doc.append(element)
    

In [15]:
corpus["one_gram_doc"] = one_gram_doc

## Bigram

In [16]:
tfidfvectorizer2 = TfidfVectorizer(stop_words=stopword_list,tokenizer = tokenizer.tokenize,
                            ngram_range=(2, 2), max_df=0.95, min_df=0.001, max_features=None)
tfid_train2 = tfidfvectorizer2.fit(corpus['nsw_token'])
tfid_bag_of_words2 = tfidfvectorizer2.transform(corpus['nsw_token'])
tfid_bigram_bag2 = tfidfvectorizer2.get_feature_names()
tfid_vector2 = tfidfvectorizer2.fit_transform(corpus['nsw_token']).toarray()
tfid_bi_gram_matrix = pd.DataFrame(tfid_vector2,columns=tfid_bigram_bag2)


In [17]:
b = tfid_bi_gram_matrix.max()
bi_gram_bag =[]
for i in range(len(b)):
    if b[i] >= b.quantile(.45):
        bi_gram_bag.append(b.index[i])
len(bi_gram_bag)

1978

In [18]:
def to_list(row):
    transform =[]
    for i in row:
        transform.append(' '.join(list(i)))
    return transform
        

In [19]:
corpus['bigram'] = corpus.nsw_token.apply(lambda x: ngrams(x.split(), 2))
corpus['bigram'] = corpus['bigram'].apply(lambda x: list(x))
corpus['bigram'] = corpus['bigram'].apply(lambda x: to_list(x))
corpus['bigram'] =  corpus.bigram.apply(lambda x: list(set(x).intersection(bi_gram_bag)))

## Trigram

In [20]:
count_vec2 = CountVectorizer(stop_words=stopword_list,tokenizer = tokenizer.tokenize,
                            ngram_range=(3, 3), max_df=0.95, min_df=0.0015, max_features=None)
count_train2 = count_vec2.fit(content_list)
bag_of_words2 = count_vec2.transform(content_list)
trigram_bag = count_vec2.get_feature_names()
print(len(trigram_bag))
corpus['trigram'] = corpus.nsw_token.apply(lambda x: ngrams(x.split(), 3))
corpus['trigram'] = corpus['trigram'].apply(lambda x: list(x))
corpus['trigram'] = corpus.trigram.apply(lambda x: to_list(x))
corpus['trigram'] =  corpus.trigram.apply(lambda x: list(set(x).intersection(trigram_bag)))


155


/Users/ritahoang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: generator 'ngrams' raised StopIteration
  


In [21]:
corpus

,id,content,label,token_docs,unique_token_docs,nsw_token,one_gram_doc,bigram,trigram
0,tr_doc_1,two german tourists have been found safe and w...,c1,"[german, tourist, found, safe, spending, hour,...","[fell, located, rugged, hatton, safe, lost, ae...",german tourist found safe spending hour lost r...,"[german, sergeant, man, area, missing, mackay,...",[],[]
1,tr_doc_2,act police have seized a rare drug during a ra...,c1,"[act, police, seized, rare, drug, raid, florey...","[woman, number, believed, home, act, territory...",act police seized rare drug raid florey home p...,"[woman, number, home, act, territory, sergeant...",[police found],[]
2,tr_doc_3,a 50-year-old brisbane man has been charged wi...,c1,"[50-year-old, brisbane, man, charged, fraud, a...","[number, driver, car, noticing, radio, fraud, ...",50-year-old brisbane man charged fraud alleged...,"[court, number, meter, man, car, driver, polic...","[number people, taxi driver, man charged, magi...",[]
3,tr_doc_4,in-depth discussions are continuing to resolve...,c1,"[in-depth, discussion, continuing, resolve, sa...","[in-depth, fixed, item, held, service, screw, ...",in-depth discussion continuing resolve safety ...,"[fixed, item, service, screw, workshop, injury...","[safety concern, public sector, industrial rel...",[]
4,tr_doc_5,homicide detectives are still questioning a ma...,c1,"[homicide, detective, questioning, man, fatal,...","[homicide, inside, injury, nearby, worker, fat...",homicide detective questioning man fatal stabb...,"[died, station, attack, detective, attacked, a...","[man died, police station, stable condition]",[]
5,tr_doc_6,the cole royal commission has recommended crim...,c1,"[cole, royal, commission, recommended, crimina...","[volume, threat, unionist, backed, workplace, ...",cole royal commission recommended criminal cha...,"[body, criminal, parliament, business, commiss...","[health safety, tony abbott, small business, u...",[]
6,tr_doc_7,the dramatic hijacking of a turkish airliner t...,c1,"[dramatic, hijacking, turkish, airliner, athen...","[cnn-turk, hijacking, suspect, shortly, aboard...",dramatic hijacking turkish airliner athens end...,"[capital, george, athens, greek, phone, weapon...","[half hour, prime minister, foreign minister, ...",[]
7,tr_doc_8,the hijacker of a turkish plane has surrendere...,c1,"[hijacker, turkish, plane, surrendered, releas...","[body, greek, athens, involved, shortly, domes...",hijacker turkish plane surrendered releasing p...,"[body, greek, athens, domestic, flight, minist...","[terrorist organisation, 200 people, prime min...",[]
8,tr_doc_9,the judge overseeing the trial in the united s...,c1,"[judge, overseeing, trial, united, state, zach...","[overseeing, moussaoui, 11th, sceptical, conce...",judge overseeing trial united state zacharias ...,"[level, attack, court, united, judge, impose, ...",[united state],[]
9,tr_doc_10,at least 68 people have been killed and 34 inj...,c1,"[people, killed, injured, riot, attempted, jai...","[wielded, jailbreak, overcome, honduran, gun, ...",people killed injured riot attempted jailbreak...,"[died, attacked, dead, riot, officer, fire, pe...","[people killed, killed injured, set fire]",[]


## TF-IDF

In [22]:
# from collections import Counter
# # one_gram_count =Counter(one_gram)
# corpus['tf'] = corpus['token_docs'].apply(lambda row: Counter(row))


In [ ]:
corpus.to_csv("corpus")